__Merge Excel Documents__</br>
<img src="http://static1.squarespace.com/static/530979d9e4b04bff4a3aadf5/t/5446c34ce4b0e5a2c7ff2614/1413923661373/DewberryLogo_RGB.png?format=1500w" width="40%" align='right'/>
PYTHON 3.6</br>
Overview: This notebook is to combine Excel files with water surface elevations into one file.</br>
Updated 2018-12-12</br>
by Chris Maderia: cmaderia@dewberry.com <br/>
by Stephen Duncan: sduncan@dewberry.com <br/>

# Merge Excel

__May require the following:__ </br>
conda install openpyxl</br>
or</br>
pip install openpyxl</br>


In [1]:
import os
import boto3
from glob import glob
import pandas as pd
from io import StringIO, BytesIO
import openpyxl  # used for Excel Writer
s3 = boto3.resource('s3')
client = boto3.client('s3')

## Define bucket and folder path (Prefix) to where Excel WSE files are stored

#### Input working directory

In [2]:
in_bucket_name = 'probmodelingrepository'
#in_working_dir = 'Augusta-Levee-AOP/ProductionRuns/outputs'
in_working_dir = 'AR-LA-MS-River/ProductionRuns/outputs'
in_folder_name = 'BaseMiss'
in_prefix_name = os.path.join(in_working_dir, in_folder_name).replace("\\","/")
file_ext = '.xlsx'  # Excel file extension

#### Output working directory

In [3]:
out_bucket_name = 'probmodelingrepository'
out_working_dir = 'Augusta-Levee-AOP/ProductionRuns/outputs'
out_folder_name = 'Augusta_Base'
out_prefix_name = os.path.join(out_working_dir, out_folder_name).replace("\\","/")

print('In: ', in_prefix_name)
print('Out: ',out_prefix_name)

In:  AR-LA-MS-River/ProductionRuns/outputs/BaseMiss
Out:  Augusta-Levee-AOP/ProductionRuns/outputs/Augusta_Base


## Functions

### Function to check if file or directory exists

In [4]:
# http://easyprog99.blogspot.com/2017/05/boto3-check-if-s3-folder-exists.html
def fileExists(bucket_name, key, filename=None):
    client = boto3.client('s3')
    # for folders only
    if filename is None:
        result = client.list_objects_v2(Bucket=bucket_name, Prefix=key+'/')
    # for files only
    else:
        result = client.list_objects_v2(Bucket=bucket_name, Prefix=key+'/'+filename)
    exists=False
    if 'Contents' in result:
        exists=True
    return exists

### Function to get list of files using Boto3
in = (bucket name, prefix (path) name, file extension)</br>
out = list of files

In [5]:
def get_files(bucketname, prefixname, textstring):
    filelist = []
    bucket = s3.Bucket(name=bucketname)
    FilesNotFound = True
    for obj in bucket.objects.filter(Prefix=prefixname):
        if textstring in str(obj) and 'xml' not in str(obj):
            filelist.append('{0}:{1}'.format(bucket.name, obj.key))
        FilesNotFound = False
    if FilesNotFound:
        print("ALERT", "No file in {0}/{1}".format(bucket, prefixname))
    return filelist

### Function to compile all Excel files as data frames into an Ordered Dict (one key for each set of structures - market baskets, uncorrelated, uniform, etc.)
in = list of Excel files with one or more sheets (one sheet for each structure set)</br>
out = ordered dictionary of data frames (one for each structure set)

In [11]:
def compile_excel(in_xlsx):
    first_excel_file = in_xlsx[:1]
    other_excel_files = in_xlsx[1:]
    # get first Excel file as dictionary of data frames (one for each sheet)
    for res in first_excel_file:
        print(res)
        colstring = os.path.basename(res).split('_')[0].replace('-','_')
        # read the data
        obj = s3.Object(res.split(':')[0], res.split(':')[1])
        data = BytesIO(obj.get()['Body'].read())
        data.seek(0)
        # create dict of data frames
        first_dfdict = pd.read_excel(data, sheet_name=None, sep=',', encoding='utf-8')
        for sheet in first_dfdict:
            first_dfdict[str(sheet)].rename(columns={0:colstring}, inplace=True)
            
    # append the others to first_dfdict
    
    for i, res in enumerate(other_excel_files):
        if i%30==0: print(res)
        if i==len(other_excel_files)-1: print(res)
        colstring = os.path.basename(res).split('_')[0].replace('-','_')
        # read the data
        obj = s3.Object(res.split(':')[0], res.split(':')[1])
        data = BytesIO(obj.get()['Body'].read())
        data.seek(0)
        # creat dict of data frames and append
        if i == 0:
            second_dfdict = pd.read_excel(data, sheet_name=None, sep=',', encoding='utf-8')
            for sheet in second_dfdict:
                second_dfdict[str(sheet)].rename(columns={0:colstring}, inplace=True)
                second_dfdict[str(sheet)] = pd.concat([first_dfdict[str(sheet)], second_dfdict[str(sheet)]], axis=1)#, sort=False)
            base_dfdict = second_dfdict
        else:
            next_dfdict = pd.read_excel(data, sheet_name=None, sep=',', encoding='utf-8')
            for sheet in next_dfdict:
                next_dfdict[str(sheet)].rename(columns={0:colstring}, inplace=True)
                next_dfdict[str(sheet)] = pd.concat([base_dfdict[str(sheet)], next_dfdict[str(sheet)]], axis=1)#, sort=False)
            base_dfdict = next_dfdict
            
    return base_dfdict

## Run it

In [7]:
results_xlsx = get_files(in_bucket_name, in_prefix_name, file_ext)
print(results_xlsx[0])
print('Count:', len(results_xlsx))

probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E301/BaseMiss-E301_structures.xlsx
Count: 300


In [12]:
out_dfdict = compile_excel(results_xlsx)

probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E301/BaseMiss-E301_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E302/BaseMiss-E302_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E332/BaseMiss-E332_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E362/BaseMiss-E362_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E392/BaseMiss-E392_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E422/BaseMiss-E422_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E452/BaseMiss-E452_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E482/BaseMiss-E482_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/outputs/BaseMiss/E512/BaseMiss-E512_structures.xlsx
probmodelingrepository:AR-LA-MS-River/ProductionRuns/ou

In [13]:
# get the first five rows of the data frame associated with the first key in the dict
out_dfdict[list(out_dfdict.keys())[0]].head()

,BaseMiss_E301,BaseMiss_E302,BaseMiss_E303,BaseMiss_E304,BaseMiss_E305,BaseMiss_E306,BaseMiss_E307,BaseMiss_E308,BaseMiss_E309,BaseMiss_E310,...,BaseMiss_E591,BaseMiss_E592,BaseMiss_E593,BaseMiss_E594,BaseMiss_E595,BaseMiss_E596,BaseMiss_E597,BaseMiss_E598,BaseMiss_E599,BaseMiss_E600
0,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,...,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000,-9999.000000
1,180.311111,180.310135,180.312195,180.316971,180.311478,180.321625,180.319763,180.318314,180.310287,180.313263,...,180.310120,180.312271,180.311172,180.310318,180.311813,180.311188,180.316330,180.312027,180.309387,180.319824
2,164.518860,164.507782,164.500809,164.495453,164.521881,164.618317,164.568848,164.633820,164.515427,164.584976,...,167.747314,169.580292,167.129211,164.532974,165.528824,169.283676,170.861649,164.893463,168.210876,168.280914
3,145.739716,141.730331,144.122314,144.587997,145.928528,145.244202,144.618301,146.751083,146.349258,146.117783,...,161.099197,163.091003,160.425262,155.983154,158.374069,163.054764,164.425552,157.505539,161.606003,161.947235
4,129.238373,125.501053,127.657265,128.220886,129.415451,128.799927,128.229492,130.132065,129.763168,129.536606,...,144.669815,146.353668,143.929184,139.527985,141.938522,146.382202,147.327042,141.136688,144.942398,145.333221


### Write to Excel and CSV files (one sheet)

In [16]:
outfolder = os.getcwd()
for sheet in out_dfdict:
    out_dfdict[str(sheet)].to_excel(os.path.join(outfolder, str(sheet)+'.xlsx'))
    #out_dfdict[str(sheet)].to_csv(os.path.join(outfolder, str(sheet)+'.csv'))

### Write to Excel file (multiple sheets - one for each structure set)

In [17]:
writer = pd.ExcelWriter(os.path.join(os.getcwd(), 'Augusta_Base', 'Augusta.xlsx'), engine='openpyxl')

In [18]:
os.getcwd()

'C:\\Users\\sduncan\\Desktop\\probrisk\\zarr'

In [19]:
for sheet in out_dfdict:
    out_dfdict[str(sheet)].to_excel(writer, sheet_name=str(sheet))

C:\Users\sduncan\AppData\Local\Continuum\anaconda3\lib\site-packages\openpyxl\workbook\child.py:99: UserWarning: Title is more than 31 characters. Some applications may not be able to read the file
  warnings.warn("Title is more than 31 characters. Some applications may not be able to read the file")


In [20]:
writer.save()

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\sduncan\\Desktop\\probrisk\\zarr\\Augusta_Base\\Augusta.xlsx'

## Output to S3 (Testing)

#### how does this take an Excel file?